In [1]:
import os
import sys
import pandas as pd
sys.path.append('/Users/dane/src/datatools')

In [2]:
%cd /Users/dane/src/datatools/ipy/notebooks/regtest/dev
df = pd.read_csv('../data/v13_full.csv')

/Users/dane/src/datatools/ipy/notebooks/regtest/dev


In [27]:
from regtest.ingest_log_files import StrMap
pmap = StrMap.read('../../../../regtest/project_name')

# build a reverse map and shorten names to useful part
def normalize_name(s):
    # first convert backslashes
    s = s.replace('\\', '/')
    # remove everything through regress_tests to eliminate vXX part
    s = s[s.find('regress_tests')+13:]
    return s
    
# build reverse map from test number to normalized name
rmap = {v:normalize_name(k) for k, v in pmap.items()}

In [4]:
df.head(1)

,passed,build,revision,project,machine,sim_time,elapsed_time,os,aut
0,1,B8112,REV10,10226,TESTEXAMPLES,1.222,1.38702,4,1436


In looking at test times it only makes sense to look at the tests that passed as failed tests could be faster or slower depending on the type of failure

In [5]:
# eliminate failed tests
dfp = df[df.passed==1]

39375015
39294465


In [6]:
# we want to average time over different days so let's only keep the REV1 tests
dfpr1 = dfp[dfp.revision=='REV1']
len(dfpr1)

6277906

In [7]:
# sort by build number
dfpr1 = dfpr1.sort_index(by='build')

With the tests sorted by build we can get the first n by just taking the start of the dataframe

In [8]:
# next iterate through the tests and average 5 oldest sim and elapsed times
results = []
for project, records in dfpr1.groupby('project'):
    if len(records) > 15:
        x = dict(project=project)
        x['sim5_avg'] = records[0:5].sim_time.mean()
        x['elap5_avg'] = records[0:5].elapsed_time.mean()
        x['sim15_avg'] = records[0:15].sim_time.mean()
        x['elap15_avg'] = records[0:15].elapsed_time.mean()
        results.append(x)
        
results = pd.DataFrame(results)

In [9]:
results.head()

,elap15_avg,elap5_avg,project,sim15_avg,sim5_avg
0,1.376987,1.385042,10226,1.208533,1.2124
1,0.120866,0.120261,10227,0.022533,0.0278
2,0.105339,0.104709,10228,0.021133,0.0182
3,3.134533,3.159178,10229,2.970200,2.9764
4,0.645501,0.651209,10230,0.471067,0.4700


In [10]:
df4 = pd.read_csv('../data/v14_b8600_b8658.csv')

Now read in the v14 tests and sort in reverse order to get the last tests

In [12]:
# eliminate failed tests
print(len(df4))
dfp4 = df4[df4.passed==1]
print(len(dfp4))
dfp4r1 = dfp4[dfp4.revision=='REV1']
len(dfp4r1)

6823981
6794405


1151175

In [13]:
# sort by build number
dfp4r1 = dfp4r1.sort_index(by='build', ascending=False)

In [14]:
r14 = []
for project, records in dfp4r1.groupby('project'):
    if len(records) > 15:
        x = dict(project=project)
        x['sim5_avg4'] = records[0:5].sim_time.mean()
        x['elap5_avg4'] = records[0:5].elapsed_time.mean()
        x['sim15_avg4'] = records[0:15].sim_time.mean()
        x['elap15_avg4'] = records[0:15].elapsed_time.mean()
        r14.append(x)
        
r14 = pd.DataFrame(r14)

In [15]:
r14.head()

,elap15_avg4,elap5_avg4,project,sim15_avg4,sim5_avg4
0,1.491520,1.493294,0,1.315533,1.3184
1,0.139223,0.139271,1,0.034400,0.0344
2,0.124512,0.122585,2,0.032067,0.0298
3,3.757507,3.742130,3,3.578400,3.5846
4,0.706162,0.684658,4,0.582200,0.5636


In [24]:
results.project.describe()

count    18886.000000
mean     23830.927301
std       9395.456221
min      10226.000000
25%      14947.250000
50%      19668.500000
75%      32896.750000
max      38181.000000
Name: project, dtype: float64

it's a little confusing that v13 is in results and v14 in v14.  I should go back and rename that when I'm next working on this.

In [25]:
r14.project.describe()

count    10198.000000
mean      5099.494803
std       2944.083165
min          0.000000
25%       2550.250000
50%       5099.500000
75%       7648.750000
max      10202.000000
Name: project, dtype: float64

We can't merge on project because they are unique (names contain v13 vs v14) so we need to add the normalized names and merge on that

In [33]:
# add the names to the dfs so that they can be merged
results['name'] = results.project.apply(lambda x: rmap[x])
r14['name'] = r14.project.apply(lambda x: rmap[x])

In [34]:
combined = pd.merge(results, r14, on='name')
combined.head()

,elap15_avg,elap5_avg,project_x,sim15_avg,sim5_avg,name,elap15_avg4,elap5_avg4,project_y,sim15_avg4,sim5_avg4
0,8.271966,9.015938,10714,0.013067,0.0132,/license/features/EM_ana_100/analyst_3d_editor...,6.504271,6.439446,488,0.013400,0.0132
1,2.865297,2.864628,10715,0.028067,0.0282,/license/features/EM_ana_100/simple_analyst.emp,2.970731,2.983960,489,0.030533,0.0292
2,2.379429,2.377258,10722,0.025933,0.0256,/license/features/EM_tok_x2/simple_axiem.emp,2.414121,2.428602,495,0.027067,0.0264
3,2.379217,2.396060,10723,0.026200,0.0282,/license/features/EM_tok_x2/simple_axiem_2_lic...,2.425837,2.421400,496,0.027267,0.0264
4,1.988121,1.969902,10725,0.035333,0.0370,/license/features/EM_tok_x2/simple_axiem_check...,1.991665,2.004662,497,0.026267,0.0282


In [39]:
combined['delta5sim'] = combined.sim5_avg4 - combined.sim5_avg
combined['pct5sim'] = (combined.delta5sim / combined.sim5_avg) * 100
combined[['name', 'pct5sim', 'delta5sim']].sort_index(by='delta5sim', ascending=False).head(10)

,name,pct5sim,delta5sim
544,/performance/Hspice/Many_Hspice.emp,47.953551,16.9394
17706,/performance/VSS/TD/NL_S_ENV_tt21065_Parametiz...,43.299108,10.9138
545,/performance/Hspice/one_Hspice.emp,53.886120,10.6618
509,/performance/axiem/Passivity/curved_lines_thic...,36.074242,7.9144
230,/performance/axiem/Avago/Avago_stages.emp,77.614173,6.6416
513,/performance/axiem/Passivity/curved_lines_thin...,54.135946,6.5642
511,/performance/axiem/Passivity/curved_lines_thin...,52.179411,6.1076
512,/performance/axiem/Passivity/curved_lines_thin...,23.567967,5.9618
510,/performance/axiem/Passivity/curved_lines_thin...,23.669301,5.8404
13333,/buildtests/Samuils/Broadside Coupled Striplin...,175.559758,5.7552


These results include more test suites, we can pull out just the buildtest ones

In [47]:
build = combined[combined.name.apply(lambda x: x.startswith('/buildtest'))]

In [52]:
build.delta5sim.sum()

303.0992

In [51]:
len(build[build.delta5sim>0])

elap15_avg     7832
elap5_avg      7832
project_x      7832
sim15_avg      7832
sim5_avg       7832
name           7832
elap15_avg4    7832
elap5_avg4     7832
project_y      7832
sim15_avg4     7832
sim5_avg4      7832
delta5sim      7832
pct5sim        7832
dtype: int64

In [53]:
dfpr1.head()

,passed,build,revision,project,machine,sim_time,elapsed_time,os,aut
5415935,1,B8000,REV1,11290,TEST3N,0.017,0.476976,4,448
5415322,1,B8000,REV1,19795,TEST3M,0.031,1.444510,4,450
5415321,1,B8000,REV1,19794,TEST3M,0.044,2.466660,4,450
5415320,1,B8000,REV1,19793,TEST3M,0.034,1.466090,4,450
5415319,1,B8000,REV1,19792,TEST3M,0.026,1.441700,4,450


We can look at a specific build/ref to get the complete test suite for that build and look at it's times.

In [54]:
b8000 = dfpr1[dfpr1.build == 'B8000']

In [55]:
len(b8000)

19390

In [56]:
b8658 = dfp4r1[dfp4r1.build == 'B8658']

In [57]:
len(b8658)

19223

In [60]:
b8000.elapsed_time.sum()

40120.989794900001

In [61]:
b8658.elapsed_time.sum()

29256.769313200002

In [64]:
b8000['name'] = b8000.project.apply(lambda x: rmap[x])
b8658['name'] = b8658.project.apply(lambda x: rmap[x])

/Users/dane/env/dt/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/dane/env/dt/lib/python3.4/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [65]:
b8000b = b8000[b8000.name.apply(lambda x: x.startswith('/buildtest'))]
b8658b = b8658[b8658.name.apply(lambda x: x.startswith('/buildtest'))]
print(len(b8000b))
print(len(b8658b))

10350
10825


In [68]:
b8000b.sim_time.sum()

4032.4000000000001

In [69]:
b8658b.sim_time.sum()

4493.5439999999999